In [57]:
import pandas as pd
import numpy as np
from IKS import IKS
from detectors import IBDD
from sklearn.ensemble import RandomForestClassifier

In [58]:
reference = pd.read_csv("Datasets/2CDT.txt", header=None, index_col=False,sep=',')
window_full = pd.read_csv("Datasets/2CHT.txt", header=None, index_col=False,sep=',')
reference

,0,1,2
0,-2.173800,-0.30919,1
1,-0.676320,1.07960,1
2,3.939200,2.07350,2
3,0.865890,1.83250,1
4,0.812800,0.51703,1
...,...,...,...
15995,-0.040798,-0.51266,1
15996,1.958700,1.39020,1
15997,4.399500,5.91600,2
15998,4.364600,3.11840,1


In [59]:
window_full

,0,1,2
0,-0.896260,0.99768,1
1,0.076403,1.27380,1
2,0.344320,0.17757,1
3,-0.654850,1.94540,1
4,-0.066363,-1.56120,1
...,...,...,...
15995,5.291700,1.87620,1
15996,2.664000,6.38980,1
15997,6.651200,1.25040,2
15998,1.714700,2.07200,1


In [60]:
reference[2].replace(1, int(0), inplace=True)
reference[2].replace(2, int(1), inplace=True)

window_full[2].replace(1, int(0), inplace=True)
window_full[2].replace(2, int(1), inplace=True)
# reference = reference.map(lambda x: 0 if 1 else 1)
# window = window.map(lambda x: 0 if 1 else 1)

In [61]:
reference

,0,1,2
0,-2.173800,-0.30919,0
1,-0.676320,1.07960,0
2,3.939200,2.07350,1
3,0.865890,1.83250,0
4,0.812800,0.51703,0
...,...,...,...
15995,-0.040798,-0.51266,0
15996,1.958700,1.39020,0
15997,4.399500,5.91600,1
15998,4.364600,3.11840,0


In [62]:
window_full

,0,1,2
0,-0.896260,0.99768,0
1,0.076403,1.27380,0
2,0.344320,0.17757,0
3,-0.654850,1.94540,0
4,-0.066363,-1.56120,0
...,...,...,...
15995,5.291700,1.87620,0
15996,2.664000,6.38980,0
15997,6.651200,1.25040,1
15998,1.714700,2.07200,0


In [63]:
X_train = reference.iloc[:, :-1]
X_test = reference.iloc[:, -1]
y_train = window_full.iloc[:, :-1]
y_test = window_full.iloc[:, -1]

clf = RandomForestClassifier(n_estimators=200)
clf.fit(X_train, X_test)

C:\AnacondaP\Lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
C:\AnacondaP\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
C:\AnacondaP\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
C:\AnacondaP\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
C:\AnacondaP\Lib\site-pac

RandomForestClassifier(n_estimators=200)

In [64]:
labels_predict = []
i = 0
while i < 300:
    labels_predict.append(int(clf.predict([y_train.iloc[i, :]])))
    i+=1

C:\Users\Luiz Fernando\AppData\Local\Temp\ipykernel_9524\2979722886.py:4: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  labels_predict.append(int(clf.predict([y_train.iloc[i, :]])))
C:\Users\Luiz Fernando\AppData\Local\Temp\ipykernel_9524\2979722886.py:4: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  labels_predict.append(int(clf.predict([y_train.iloc[i, :]])))
C:\Users\Luiz Fernando\AppData\Local\Temp\ipykernel_9524\2979722886.py:4: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated 

C:\Users\Luiz Fernando\AppData\Local\Temp\ipykernel_9524\2979722886.py:4: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  labels_predict.append(int(clf.predict([y_train.iloc[i, :]])))
C:\Users\Luiz Fernando\AppData\Local\Temp\ipykernel_9524\2979722886.py:4: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  labels_predict.append(int(clf.predict([y_train.iloc[i, :]])))
C:\Users\Luiz Fernando\AppData\Local\Temp\ipykernel_9524\2979722886.py:4: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated 

C:\Users\Luiz Fernando\AppData\Local\Temp\ipykernel_9524\2979722886.py:4: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  labels_predict.append(int(clf.predict([y_train.iloc[i, :]])))
C:\Users\Luiz Fernando\AppData\Local\Temp\ipykernel_9524\2979722886.py:4: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  labels_predict.append(int(clf.predict([y_train.iloc[i, :]])))
C:\Users\Luiz Fernando\AppData\Local\Temp\ipykernel_9524\2979722886.py:4: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated 

C:\Users\Luiz Fernando\AppData\Local\Temp\ipykernel_9524\2979722886.py:4: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  labels_predict.append(int(clf.predict([y_train.iloc[i, :]])))
C:\Users\Luiz Fernando\AppData\Local\Temp\ipykernel_9524\2979722886.py:4: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  labels_predict.append(int(clf.predict([y_train.iloc[i, :]])))
C:\Users\Luiz Fernando\AppData\Local\Temp\ipykernel_9524\2979722886.py:4: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated 

C:\Users\Luiz Fernando\AppData\Local\Temp\ipykernel_9524\2979722886.py:4: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  labels_predict.append(int(clf.predict([y_train.iloc[i, :]])))
C:\Users\Luiz Fernando\AppData\Local\Temp\ipykernel_9524\2979722886.py:4: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  labels_predict.append(int(clf.predict([y_train.iloc[i, :]])))
C:\Users\Luiz Fernando\AppData\Local\Temp\ipykernel_9524\2979722886.py:4: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated 

...

C:\Users\Luiz Fernando\AppData\Local\Temp\ipykernel_9524\2979722886.py:4: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  labels_predict.append(int(clf.predict([y_train.iloc[i, :]])))
C:\Users\Luiz Fernando\AppData\Local\Temp\ipykernel_9524\2979722886.py:4: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  labels_predict.append(int(clf.predict([y_train.iloc[i, :]])))
C:\Users\Luiz Fernando\AppData\Local\Temp\ipykernel_9524\2979722886.py:4: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated 

C:\Users\Luiz Fernando\AppData\Local\Temp\ipykernel_9524\2979722886.py:4: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  labels_predict.append(int(clf.predict([y_train.iloc[i, :]])))
C:\Users\Luiz Fernando\AppData\Local\Temp\ipykernel_9524\2979722886.py:4: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  labels_predict.append(int(clf.predict([y_train.iloc[i, :]])))
C:\Users\Luiz Fernando\AppData\Local\Temp\ipykernel_9524\2979722886.py:4: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated 

C:\Users\Luiz Fernando\AppData\Local\Temp\ipykernel_9524\2979722886.py:4: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  labels_predict.append(int(clf.predict([y_train.iloc[i, :]])))
C:\Users\Luiz Fernando\AppData\Local\Temp\ipykernel_9524\2979722886.py:4: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  labels_predict.append(int(clf.predict([y_train.iloc[i, :]])))
C:\Users\Luiz Fernando\AppData\Local\Temp\ipykernel_9524\2979722886.py:4: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated 

C:\Users\Luiz Fernando\AppData\Local\Temp\ipykernel_9524\2979722886.py:4: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  labels_predict.append(int(clf.predict([y_train.iloc[i, :]])))
C:\Users\Luiz Fernando\AppData\Local\Temp\ipykernel_9524\2979722886.py:4: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  labels_predict.append(int(clf.predict([y_train.iloc[i, :]])))
C:\Users\Luiz Fernando\AppData\Local\Temp\ipykernel_9524\2979722886.py:4: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated 

C:\Users\Luiz Fernando\AppData\Local\Temp\ipykernel_9524\2979722886.py:4: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  labels_predict.append(int(clf.predict([y_train.iloc[i, :]])))
C:\Users\Luiz Fernando\AppData\Local\Temp\ipykernel_9524\2979722886.py:4: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  labels_predict.append(int(clf.predict([y_train.iloc[i, :]])))
C:\Users\Luiz Fernando\AppData\Local\Temp\ipykernel_9524\2979722886.py:4: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated 

In [65]:
labels_predict

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,


In [66]:
window = list(window_full.iloc[:300, :][2])
window

[0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
